# A heuristic value model
This model is meant to jump-start the learning process and advise MCTS on its very first self-play

In [1]:
BOARD_SIZE=15

In [2]:
from aegomoku.policies.heuristic_value_model import HeuristicValueModel
import tensorflow as tf
import numpy as np

In [3]:
model = HeuristicValueModel(BOARD_SIZE, value_gauge=1.0, value_stretch=.02, kappa_d=2, kappa_s=5, current_advantage=.1, bias=-0.5)

In [22]:
from aegomoku.gomoku_board import GomokuBoard

stones = "C11F9E9G8F7G7G9H8G6I8J8I9J10I6"
board = GomokuBoard(board_size=BOARD_SIZE, stones = stones)
board.plot()
model.evaluate(board.math_rep)

                                                     
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
11    .  .  X  .  .  .  .  .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  .  .  X  .  .  .  .  .    
 9    .  .  .  .  X  O  X  .  O  .  .  .  .  .  .    
 8    .  .  .  .  .  .  O  O  O  X  .  .  .  .  .    
 7    .  .  .  .  .  X  O  .  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  X  . [O] .  .  .  .  .  .    
 5    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 4    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 3    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 1    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
                                                     
      A  B  C  D  E  F  G  H  I  J  K  L  M  N  O


-0.45690027

In [23]:
from aegomoku.utils import expand
state = expand(board.math_rep)
model.detector(state).shape

TensorShape([1, 17, 17, 40])

In [24]:
diff = tf.keras.layers.Conv2D(
    name="sum_d",
    filters=1,
    kernel_size=1,
    kernel_initializer=tf.constant_initializer(
        [1.1, -1]), # current player's advantage
    bias_initializer=tf.constant_initializer(0.),
    padding='same',
    trainable=False)

peel = tf.keras.layers.Conv2D(
    filters=1, kernel_size=(3, 3),
    kernel_initializer=tf.constant_initializer([
        [[0., 0., 0.], [0., 1., 0.], [0., 0., 0.]]
    ]),
    bias_initializer=tf.constant_initializer(0.),
    trainable=False)


In [25]:
kappa_s = 5
self = model
y = self.detector(state)
y = tf.math.pow(y, kappa_s)
y = self.sum_s(y)
y = tf.math.pow(y, 2 / 5)
y = self.sum_d(y)
d = peel(diff(y))

In [26]:
from aegomoku.tools import print_channels

print_channels(((y.numpy())*10).astype(np.uint), combine=False)

shape: (17, 17, 2)
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  5  0  0  0  2  0  0  0  2  0  0]
 [ 0 22  0  6  0  6  0  5  0  2  3  2  0  2  0  0  0]
 [ 0  0 29  4  6  3  0  2  7  0 25  0  2  0  2  0  0]
 [ 0  2  5  0  3 10  3  8  0  8 29  2  0  3  0  0  0]
 [ 0  0  3 26 29  4  9  5  8  4  0  4  7  3  2  0  0]
 [ 0  4  2  6 32  0  0  0  0  0 34  8  0  0  0  0  0]
 [ 0  0  0  3  4 39 29  0  0  0  0  2  6  2  2  0  0]
 [ 0  0  2  8  2 10  0  0  2  4 29  2  0  3  0  0  0]
 [ 0  0  3  2  6  9  6  0  6  0 22  0  2  0  2  0  0]
 [ 0  2  0  2  2  2  6  5 34  0  3  0  0  2  0  0  0]
 [ 0  0  0  2  0  3  5  2  0 29  2  0  0  0  2  0  0]
 [ 0  0  2  0  3  0  2  2  0  0 22  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  2  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0

In [27]:
board.plot()

                                                     
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
11    .  .  X  .  .  .  .  .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  .  .  X  .  .  .  .  .    
 9    .  .  .  .  X  O  X  .  O  .  .  .  .  .  .    
 8    .  .  .  .  .  .  O  O  O  X  .  .  .  .  .    
 7    .  .  .  .  .  X  O  .  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  X  . [O] .  .  .  .  .  .    
 5    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 4    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 3    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 1    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
                                                     
      A  B  C  D  E  F  G  H  I  J  K  L  M  N  O


In [19]:
stones = "C11F9E9G8F7G7G9H8G6i8J8I9"
board = GomokuBoard(board_size=BOARD_SIZE, stones = stones)
board.plot()
model.evaluate(board.math_rep)

                                                     
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
11    .  .  X  .  .  .  .  .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 9    .  .  .  .  X  O  X  . [O] .  .  .  .  .  .    
 8    .  .  .  .  .  .  O  O  O  X  .  .  .  .  .    
 7    .  .  .  .  .  X  O  .  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  X  .  .  .  .  .  .  .  .    
 5    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 4    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 3    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 1    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
                                                     
      A  B  C  D  E  F  G  H  I  J  K  L  M  N  O


-0.17040965